In [ ]:
# IMPORTS AND INITIAL SETUP ALONG WITH LOADING DATASET

In [ ]:
import torch
from torchvision import datasets, transforms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision.models as models

# Checking if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations are defined
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizing the data
])

# Augmentations for training data
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-100 dataset
train_data = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_data = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

device

100%|██████████| 169001437/169001437 [00:13<00:00, 12849197.73it/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


device(type='cuda')

In [ ]:
# DATA PREPARATION

In [ ]:
from torch.utils.data import random_split

# Splitting the training data into training and validation sets
num_train = int(len(train_data) * 0.8)
num_val = len(train_data) - num_train
train_dataset, val_dataset = random_split(train_data, [num_train, num_val])

# DataLoader for the validation set
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=8, pin_memory=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [1]:
# EARLY STOPPING CLASS:- The `EarlyStopping` class stops training if the validation loss doesn't get better after a predetermined number of epochs, preventing overfitting. By stopping training at an appropriate time, this method reduces the amount of computational resources used while improving the model's generalization on new, unknown data.

In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        """
        Args:
            patience (int): How many epochs to wait after last time validation accuracy improved.
                            Default: 5
            verbose (bool): If True, prints a message for each validation accuracy improvement.
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_acc_max = float('-inf')

    def __call__(self, val_accuracy, model):
        score = val_accuracy

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_accuracy, model)
        elif score <= self.best_score:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_accuracy, model)
            self.counter = 0

    def save_checkpoint(self, val_accuracy, model):
        '''Saves model when validation accuracy increase.'''
        if self.verbose:
            print(f'Validation accuracy increased ({self.val_acc_max:.6f} --> {val_accuracy:.6f}).  Saving model...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_acc_max = val_accuracy

In [ ]:
early_stopping = EarlyStopping(patience=5, verbose=True)
def evaluate(model, loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f'Test Accuracy: {accuracy:.2f}%')
        return accuracy

def train_and_validate_model(model, train_loader, val_loader, num_epochs):
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        total = 0
        correct = 0
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)


            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = 100 * correct / total
        val_acc = evaluate(model, val_loader)  # Evaluate on validation set

        early_stopping(val_acc, model)
        if early_stopping.early_stop:
            print("Early stopping")
            break

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

In [ ]:
# CNN MODEL FOR CIPHAR-100

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Sequential container for convolutional layers utilizing max pooling to reduce spatial dimensions after batch normalization and ReLU activation
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        # Sequential container with dropout and ReLU activation for completely linked layers
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(512 * 2 * 2, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 100)  # 100 classes in CIFAR-100
        )

    def forward(self, x):
        # Apply the Conv layers
        x = self.conv_layer(x)
        # Flatten the output for the layers which are fully connected
        x = x.view(x.size(0), -1)
        # FC layers are applied
        x = self.fc_layer(x)
        return x

# Model to device
cnn_model = CNN().to(device)
print(cnn_model)

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 2

In [ ]:
criterion = nn.CrossEntropyLoss()  #setting up a loss function and optimizer for training a CNN model
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

In [ ]:
# MODEL TRAINING AND VALIDATION

In [ ]:
train_and_validate_model(cnn_model, train_loader, val_loader, 50)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Test Accuracy: 5.01%
Validation accuracy increased (-inf --> 5.010000).  Saving model...
Epoch [1/50], Loss: 4.2750, Train Acc: 3.71%, Val Acc: 5.01%
Test Accuracy: 8.46%
Validation accuracy increased (5.010000 --> 8.460000).  Saving model...
Epoch [2/50], Loss: 3.9784, Train Acc: 6.92%, Val Acc: 8.46%
Test Accuracy: 11.39%
Validation accuracy increased (8.460000 --> 11.390000).  Saving model...
Epoch [3/50], Loss: 3.7873, Train Acc: 10.06%, Val Acc: 11.39%
Test Accuracy: 14.68%
Validation accuracy increased (11.390000 --> 14.680000).  Saving model...
Epoch [4/50], Loss: 3.5888, Train Acc: 13.57%, Val Acc: 14.68%
Test Accuracy: 18.60%
Validation accuracy increased (14.680000 --> 18.600000).  Saving model...
Epoch [5/50], Loss: 3.3811, Train Acc: 17.54%, Val Acc: 18.60%
Test Accuracy: 21.39%
Validation accuracy increased (18.600000 --> 21.390000).  Saving model...
Epoch [6/50], Loss: 3.1732, Train Acc: 21.45%, Val Acc: 21.39%
Test Accuracy: 26.14%
Validation accuracy increased (21.39000

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False) #First layer of convolution with a certain stride
        self.bn1 = nn.BatchNorm2d(out_channels) #Batch normalization after the first convolution
        self.relu = nn.ReLU(inplace = True) #ReLU activation function
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False) #Second layer of convolution with stride 1, without bias
        self.bn2 = nn.BatchNorm2d(out_channels) #Batch normalization
        self.downsample = downsample #downsample to adjust channels and dimensions

    def forward(self, x):
        residual = x
        out = self.conv1(x) #convulation 1 operation
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out) #convulation 2 operation
        out = self.bn2(out)

        if self.downsample:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)
        return out

In [2]:
# RESNET MODEL FOR CIPHAR-100

In [ ]:
class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        # Set the initial convolution, batch normalization, and input channel initialization
        self.in_channels = 64
        self.conv = nn.Conv2d(3, 64, kernel_size=3, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        # Layers of residual blocks are defined
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1)) # Pooling that adapts to output size
        self.fc = nn.Linear(512, 100) # Final linear layer for classification

    def _make_layer(self, out_channels, blocks, stride):
        downsample = None  # If necessary, configure the downsample for the remaining connection.
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        layers = []

        # Creating blocks of residual layers
        layers.append(ResidualBlock(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):  # Activation, normalization, and initial convolution
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [ ]:
resnet_model = ResNet().to(device)
optimizer = optim.Adam(resnet_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print(resnet_model)

ResNet(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=

In [ ]:
train_and_validate_model(resnet_model, train_loader, val_loader, 50)  # model training and validation

Test Accuracy: 20.08%
Validation accuracy increased (-inf --> 20.080000).  Saving model...
Epoch [1/50], Loss: 3.4678, Train Acc: 16.29%, Val Acc: 20.08%
Test Accuracy: 23.60%
Validation accuracy increased (20.080000 --> 23.600000).  Saving model...
Epoch [2/50], Loss: 3.2092, Train Acc: 21.04%, Val Acc: 23.60%
Test Accuracy: 28.74%
Validation accuracy increased (23.600000 --> 28.740000).  Saving model...
Epoch [3/50], Loss: 2.8816, Train Acc: 27.28%, Val Acc: 28.74%
Test Accuracy: 33.22%
Validation accuracy increased (28.740000 --> 33.220000).  Saving model...
Epoch [4/50], Loss: 2.6505, Train Acc: 32.20%, Val Acc: 33.22%
Test Accuracy: 35.47%
Validation accuracy increased (33.220000 --> 35.470000).  Saving model...
Epoch [5/50], Loss: 2.4525, Train Acc: 36.23%, Val Acc: 35.47%
Test Accuracy: 37.06%
Validation accuracy increased (35.470000 --> 37.060000).  Saving model...
Epoch [6/50], Loss: 2.2686, Train Acc: 40.58%, Val Acc: 37.06%
Test Accuracy: 39.60%
Validation accuracy increased

In [ ]:
# VGG MODEL FOR CIPHAR-100

In [ ]:
class VGG16CIFAR100(nn.Module):
    def __init__(self):
        super(VGG16CIFAR100, self).__init__()
        # Setting up VGG-like layers with max pooling ('M') after convolutions
        self.features = self._make_layers([
            64, 64, 'M',      # 'M' stands for MaxPool
            128, 128, 'M',
            256, 256, 256, 'M',
            512, 512, 512, 'M',
            512, 512, 512, 'M'
        ])
        # Fully connected layers are used for classification, while dropout is used for regularization in the classifier.
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, 100)  # CIFAR-100 has 100 classes
        )

    def forward(self, x):
        x = self.features(x) # Through feature extractor input is passed and output is flattend in order to feed into the classifier
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3  # Input channels (RGB)
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)]
                in_channels = x
        return nn.Sequential(*layers)

In [ ]:
vgg_model = VGG16CIFAR100().to(device)  # Set up the VGG16 model that was modified to work with the CIFAR-100 dataset
print(vgg_model)

VGG16CIFAR100(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2

In [ ]:
train_and_validate_model(vgg_model, train_loader, val_loader, 50)  # model training and validation

Test Accuracy: 1.14%
Validation accuracy increased (-inf --> 1.140000).  Saving model...
Epoch [1/50], Loss: 4.6330, Train Acc: 1.00%, Val Acc: 1.14%
Test Accuracy: 1.16%
Validation accuracy increased (1.140000 --> 1.160000).  Saving model...
Epoch [2/50], Loss: 4.6105, Train Acc: 1.00%, Val Acc: 1.16%
Test Accuracy: 0.99%
EarlyStopping counter: 1 out of 5
Epoch [3/50], Loss: 4.6105, Train Acc: 1.08%, Val Acc: 0.99%
Test Accuracy: 0.96%
EarlyStopping counter: 2 out of 5
Epoch [4/50], Loss: 4.6106, Train Acc: 1.06%, Val Acc: 0.96%
Test Accuracy: 1.13%
EarlyStopping counter: 3 out of 5
Epoch [5/50], Loss: 4.6108, Train Acc: 1.02%, Val Acc: 1.13%
Test Accuracy: 1.09%
EarlyStopping counter: 4 out of 5
Epoch [6/50], Loss: 4.6107, Train Acc: 0.99%, Val Acc: 1.09%
Test Accuracy: 1.24%
Validation accuracy increased (1.160000 --> 1.240000).  Saving model...
Epoch [7/50], Loss: 4.6110, Train Acc: 1.02%, Val Acc: 1.24%
Test Accuracy: 1.18%
EarlyStopping counter: 1 out of 5
Epoch [8/50], Loss: 4.6

In [ ]:
# Convert to numpy arrays for compatibility with scikit-learn
def convert_to_numpy(dataset):
    images = []
    labels = []
    for image, label in dataset:
        image = image.permute(1, 2, 0).numpy()  # Reorder dimensions to HWC
        image = image.flatten()  # Flatten the image
        images.append(image)
        labels.append(label)
    return np.array(images), np.array(labels)

X_train, y_train = convert_to_numpy(train_data)
X_test, y_test = convert_to_numpy(test_data)

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [3]:
# SVM MODEL TRAINING FOR CIPHAR-100

In [ ]:
# Function to train and evaluate SVM
def train_svm(kernel_type):
    print(f"Training SVM with {kernel_type} kernel...")
    svm_model = svm.SVC(kernel=kernel_type)
    svm_model.fit(X_train_scaled, y_train)

    # Predict and evaluate on the validation set
    y_val_pred = svm_model.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"Validation accuracy for {kernel_type} kernel: {val_accuracy:.4f}")

# Train SVM with linear kernel
train_svm('linear')

# Train SVM with radial kernel
train_svm('rbf')

Training SVM with linear kernel...
Validation accuracy for linear kernel: 0.0856
Training SVM with rbf kernel...
